In [422]:
import math
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

The Libor rate r(s) for maturity s years is given by the Nelson-Siegel model as:

$r(s) = B0 + B1*(tau/s)*(1-exp(-s/tau)) + B2*((tau/s)*(1-exp(-s/tau)) - exp(-s/tau))$

From this, we compute the discount factor $Z(0, s) = exp(-r(s) * s)$

In [423]:
def r(s, B0=0.0408, B1=-0.0396, B2=-0.0511, tau=1.614):
    if s == 0:
        return 0

    term_1 = B0
    term_2 = B1*(tau/s)*(1-np.exp(-s/tau))
    term_3 = B2*((tau/s)*(1-np.exp(-s/tau)) - np.exp(-s/tau))

    return term_1 + term_2 + term_3

In [200]:
def Z(s, B0=0.0408, B1=-0.0396, B2=-0.0511, tau=1.614):
    return np.exp(-r(s, B0, B1, B2, tau) * s)

### Calculation of survival probabilities Q(t)

In [424]:
def weekend_rollover(w_date):
    # If payment date falls on a weekday, roll it over to Monday.
    if w_date.weekday() == 5:
        w_date += relativedelta(days=2)
    elif w_date.weekday() == 6:
        w_date += relativedelta(days=1)

    return w_date

In [425]:
payment_period = 0.25 # 4 payments per year

# maturity dates for the known spreads, as calculated on February 28, 2014 
spread_maturity_dates_before_adjustment = [
    date(2014, 9, 20),
    date(2015, 3, 20),
    date(2016, 3, 20),
    date(2017, 3, 20),
    date(2018, 3, 20),
    date(2019, 3, 20),
    date(2021, 3, 20),
    date(2024, 3, 20),
    date(2034, 3, 20),
    date(2044, 3, 20)
]

spread_maturity_dates = [ weekend_rollover(mat_date) for mat_date in spread_maturity_dates_before_adjustment ]
# spread data: {time(in years): spread (bps)}
spread_data = {
    0.5: 103.07,
    1: 104.68,
    2: 106.74,
    3: 110.31,
    4: 113.38,
    5: 116.98,
    7: 128.78,
    10: 143.51,
    20: 159.43,
    30: 164.13
}

R = 0.45 #Recovery rate for standard CDS

Payment convention assumed is (Actual / 360).

If payment dates fall on weekend, they are rolled over to Monday.

h(s), which is the instantaneous forward default rate, is assumed to be piecewise constant between the different maturities for which we know the spreads.

If we can calibrate the values of this instantaneous forward default rate, we can get the survival curve.

$-\ln(Q(t)) = \int_0^t h(s) \,ds$

In [426]:
# Payment dates for standard CDS = 20 March, 20 June, 20 September, 20 December
# Payment dates for non-standard CDS = 15 February, 15 May, 15 August, 15 November
standard_cds_payment_dates = [date(2014, 3, 20), date(2014, 6, 20), date(2014, 9, 20), date(2014, 12, 20)] # year is irrelevant
nonstandard_cds_payment_dates = [date(2014, 2, 15), date(2014, 5, 15), date(2014, 8, 15), date(2014, 11, 15)] # year is irrelevant

def next_payment_date(current_date, payment_dates=standard_cds_payment_dates):
    payment_date = current_date

    if current_date.month < payment_dates[0].month:
        payment_date = date(current_date.year, payment_dates[0].month, payment_dates[0].day)
    elif current_date.month > payment_dates[0].month and current_date.month < payment_dates[1].month:
        payment_date = date(current_date.year, payment_dates[1].month, payment_dates[1].day)
    elif current_date.month > payment_dates[1].month and current_date.month < payment_dates[2].month:
        payment_date = date(current_date.year, payment_dates[2].month, payment_dates[2].day)
    elif current_date.month > payment_dates[2].month and current_date.month < payment_dates[3].month:
        payment_date = date(current_date.year, payment_dates[3].month, payment_dates[3].day)
    else:
        # date belongs to one of the payment months
        if current_date.day < payment_dates[0].day:
            payment_date = date(current_date.year, current_date.month, payment_dates[0].day)
        else:
            payment_date = next_payment_date(current_date + relativedelta(months=1))

    # If payment date falls on a weekday, roll it over to Monday.
    payment_date = weekend_rollover(payment_date)

    return payment_date

In [427]:
contract_date = date(2014, 2, 28)
effective_date = contract_date + relativedelta(days=1)

In [428]:
def K(T_i, m=12):
    return math.ceil(m*T_i)

def s(k, T_i, m=12):
    # We are choosing m=12, because we want to calculate the value in monthly subintervals.
    return k*T_i / K(T_i, m)

In [387]:
# def cds_mtm_value(h_i, end_date, prev_h={}):
#     a = (1-R)/2
#     b = 0

#     effective_date = contract_date + relativedelta(days=1)
#     maturity_date = next_payment_date(date(2014, 9, 1))

#     T_1 = (maturity_date - effective_date).days / 360
    
#     for k in range(K(T_1)):
#         b += (Z(s(k, T_1)) + Z(s(k+1, T_1))) * (np.exp(-s(k, T_1)*h_i) - np.exp(-s(k+1, T_1)*h_i))
#         print(f"s:{s(k, T_1)}, h:{h_i}")


#     c = 0.5 * spread_data[0.5] / 100 # Dividing by 100 to convert bps to %.
#     d = 0

#     current_date = effective_date
#     next_pay_date = next_payment_date(effective_date)

#     # Adding two days in the while loop check to account for weekend
#     while next_pay_date <= spread_maturity_dates[0] + relativedelta(days=2):
#         delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
#         t_1 = (current_date - effective_date).days / 360
#         t_2 = (next_pay_date - effective_date).days / 360

#         d += delta_t * Z(t_2) * (np.exp(-t_1*h_i) + np.exp(-t_2*h_i))
        
#         current_date = next_pay_date
#         next_pay_date = next_payment_date(next_pay_date)
    
#     print("current_date", current_date)
#     print("next_pay_date", next_pay_date)
    
#     print("a:", a)
#     print("b:", b)
#     print("c:", c)
#     print("d:", d)

#     return a*b-c*d

In [388]:
# cds_mtm_value(0.01, spread_maturity_dates[0])

In [476]:
def Q_h(t, h_i, prev_h={}):
    # t: time in years
    # h_i: instantaneous forward default rate for ith maturity

    end_date = effective_date + relativedelta(days=int(t*360))

    h_t = 0
    for idx, mat_date in enumerate(spread_maturity_dates):
        start_date = effective_date if idx==0 else spread_maturity_dates[idx-1]

        if mat_date < end_date:
            h_t += prev_h[mat_date] * ((mat_date - start_date).days / 360)
        elif mat_date >= end_date:
            h_t += h_i * ((end_date - start_date).days / 360)

            break
    
    return np.exp(-h_t)

# differential of Q_h
def diff_Q_h(t, h_i, prev_h):
    end_date = effective_date + relativedelta(days=int(t*360))
    diff = 0

    for idx, mat_date in enumerate(spread_maturity_dates):
        if mat_date >= end_date:
            start_date = effective_date if idx == 0 else spread_maturity_dates[idx-1]
            diff = (end_date - start_date).days / 360
            break

    return -diff*Q_h(t, h_i, prev_h)

In [477]:

def cds_mtm_value(h_i, maturity_date, prev_h={}):
    a = (1-R)/2
    b = 0

    T = (maturity_date - effective_date).days / 360
    
    for k in range(K(T)):
        b += (Z(s(k, T)) + Z(s(k+1, T))) * (Q_h(s(k,T), h_i, prev_h) - Q_h(s(k+1, T), h_i, prev_h))

    maturity_date_index = spread_maturity_dates.index(maturity_date)
    
    c = 0.5 * list(spread_data.values())[maturity_date_index] / 100 # Dividing by 100 to convert bps to %.
    d = 0

    current_date = effective_date
    next_pay_date = next_payment_date(effective_date)

    while next_pay_date <= maturity_date:
        delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
        t_1 = (current_date - effective_date).days / 360
        t_2 = (next_pay_date - effective_date).days / 360

        d += delta_t * Z(t_2) * (Q_h(t_1, h_i, prev_h) + Q_h(t_2, h_i, prev_h))
        
        current_date = next_pay_date
        next_pay_date = next_payment_date(next_pay_date)
    
    return a*b-c*d

In [392]:
# # differential of cds MTM value, for using in Newton-Raphson method.
# def d_cds_mtm_value(h_1=0.5):
#     a = (1-R)/2
#     b = 0

#     effective_date = contract_date + relativedelta(days=1)
#     maturity_date = next_payment_date(date(2014, 9, 1))
    
#     T_1 = (maturity_date - effective_date).days / 360

#     for k in range(K(T_1)):
#         print("diff:", s(k, T_1))
#         b += (Z(s(k, T_1)) + Z(s(k+1, T_1))) * (-s(k, T_1) * np.exp(-s(k, T_1)*h_1) + s(k+1, T_1) * np.exp(-s(k+1, T_1)*h_1))

#     c = 0.5 * spread_data[0.5] / 100 # Dividing by 100 to convert bps to %
#     d = 0

#     current_date = effective_date
#     next_pay_date = next_payment_date(effective_date)

#     while next_pay_date <= spread_maturity_dates[0] + relativedelta(days=2):
#         delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
#         t_1 = (current_date - effective_date).days / 360
#         t_2 = (next_pay_date - effective_date).days / 360
        
#         d += delta_t * Z(t_2) * (-t_1 * np.exp(-t_1*h_1) - t_2 * np.exp(-t_2*h_1))
        
#         current_date = next_pay_date
#         next_pay_date = next_payment_date(next_pay_date)

#     print("a:", a)
#     print("b:", b)
#     print("c:", c)
#     print("d:", d)
    
#     return a*b-c*d

In [383]:
# d_cds_mtm_value(0.01)

In [478]:
# differential of cds MTM value, for using in Newton-Raphson method.
def d_cds_mtm_value(h_i, maturity_date, prev_h={}):
    a = (1-R)/2
    b = 0
    
    T = (maturity_date - effective_date).days / 360

    for k in range(K(T)):
        # end_date_1 = effective_date + relativedelta(days=int(s(k, T)*360))
        # end_date_2 = effective_date + relativedelta(days=int(s(k+1, T)*360))
        # diff_1 = 0
        # diff_2 = 0

        # for idx, date in enumerate(spread_maturity_dates):
        #     diff_1_check = False
        #     diff_2_check = False

        #     if date >= end_date_1 and not diff_1_check:
        #         start_date_1 = effective_date if idx == 0 else spread_maturity_dates[idx-1]
        #         diff_1 = (end_date_1-start_date_1).days / 360
        #         diff_1_check = True

        #     if date >= end_date_2 and not diff_2_check:
        #         start_date_2 = effective_date if idx == 0 else spread_maturity_dates[idx-1]
        #         diff_2 = (end_date_2-start_date_2).days / 360
        #         diff_2_check = True

        #     if diff_1_check and diff_2_check:
        #         break
            

        b += (Z(s(k, T)) + Z(s(k+1, T))) * (diff_Q_h(s(k, T), h_i, prev_h) - diff_Q_h(s(k+1, T), h_i, prev_h))

    maturity_date_index = spread_maturity_dates.index(maturity_date)
    c = 0.5 * list(spread_data.values())[maturity_date_index] / 100 # Dividing by 100 to convert bps to %.
    d = 0

    current_date = effective_date
    next_pay_date = next_payment_date(effective_date)

    while next_pay_date <= maturity_date:
        delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
        t_1 = (current_date - effective_date).days / 360
        t_2 = (next_pay_date - effective_date).days / 360
        
        # end_date_1 = current_date
        # end_date_2 = next_pay_date
        # diff_1 = 0
        # diff_2 = 0

        # for idx, date in enumerate(spread_maturity_dates):
        #     diff_1_check = False
        #     diff_2_check = False

        #     if date >= end_date_1 and not diff_1_check:
        #         start_date_1 = effective_date if idx == 0 else spread_maturity_dates[idx-1]
        #         diff_1 = (end_date_1-start_date_1).days / 360
        #         diff_1_check = True

        #     if date >= end_date_2 and not diff_2_check:
        #         start_date_2 = effective_date if idx == 0 else spread_maturity_dates[idx-1]
        #         diff_2 = (end_date_2-start_date_2).days / 360
        #         diff_2_check = True

        #     if diff_1_check and diff_2_check:
        #         break

        d += delta_t * Z(t_2) * (diff_Q_h(t_1, h_i, prev_h) + diff_Q_h(t_2, h_i, prev_h))
        
        current_date = next_pay_date
        next_pay_date = next_payment_date(next_pay_date)
    
    return a*b-c*d

In [473]:
d_cds_mtm_value(0.01, spread_maturity_dates[0])

0.4777721806748909

In [479]:
def newton_raphson_method(f, d_f, maturity_date, prev_h={}, x=0.01, eps=1e-6):
    np.seterr(all="raise")

    val = f(x, maturity_date, prev_h)
    while abs(val) > eps:
        diff = d_f(x, maturity_date, prev_h) #differential of f(h1)

        if np.abs(diff) > np.sqrt(np.finfo(float).eps):            
            x = x - float(val) / diff
            val = f(x, maturity_date, prev_h)
        else:
            return None
        
    return x

In [480]:
h_values = {}

for mat_date in spread_maturity_dates:
    h = newton_raphson_method(cds_mtm_value, d_cds_mtm_value, mat_date, h_values)
    h_values[mat_date] = h
    
h_values

{datetime.date(2014, 9, 22): 1.9062919378974001,
 datetime.date(2015, 3, 20): 1.953364747912047,
 datetime.date(2016, 3, 21): 2.015745505817559,
 datetime.date(2017, 3, 20): 2.129558513721298,
 datetime.date(2018, 3, 20): 2.2270634200467003,
 datetime.date(2019, 3, 20): 2.340216635297322,
 datetime.date(2021, 3, 22): 2.7012838580319865,
 datetime.date(2024, 3, 20): 3.1345870523377024,
 datetime.date(2034, 3, 20): 3.5864510796618547,
 datetime.date(2044, 3, 21): 3.717296614970036}

In [454]:
def Q(t):
    # Above we have a function Q_h(t, h_i, prev_h={}) which requires t and h_i as arguments.
    # Now using the values of h we have found, this function provides value of Q at any time t years from the effective date.

    end_date = effective_date + relativedelta(days=int(t*360))

    for mat_date in spread_maturity_dates:
        if end_date <= mat_date:
            return Q_h(t, h_values[mat_date], h_values)
        
    return Q_h(t, list(h_values.values())[-1], h_values) # if given time is more than the max date we have, return last h value

In [481]:
Q_values = {}

for mat_date in spread_maturity_dates:
    Q_values[mat_date] = Q((mat_date-effective_date).days/360)
    
Q_values

{datetime.date(2014, 9, 22): 0.33772363917412773,
 datetime.date(2015, 3, 20): 0.12786455820730133,
 datetime.date(2016, 3, 21): 0.01637950744353088,
 datetime.date(2017, 3, 20): 0.0019018188778591617,
 datetime.date(2018, 3, 20): 0.00020008803450173723,
 datetime.date(2019, 3, 20): 1.853853448339687e-05,
 datetime.date(2021, 3, 22): 7.575411089732266e-08,
 datetime.date(2024, 3, 20): 5.575006107287449e-12,
 datetime.date(2034, 3, 20): 8.744229386543874e-28,
 datetime.date(2044, 3, 21): 3.5937449849945805e-44}